In [71]:
# indicate shortestServiceFirst or firstInFirstOut
servicePreference = 'firstInFirstOut'
# indicate ExponentialDistribution, LongtailDistribution or BimodalDistribution
serviceRateDistribution = 'ExponentialDistribution'
mu = 0.90 # mean service time
personnelAmount = 8 #amount of servers
interarrivalTime = 8 #denominator/divisor interarravial time
customerAmount = 239000 #  amount of customers to visit the shop
samplingSpace = customerAmount / 239 * 240 / 200
halfMargin = samplingSpace / 12

In [72]:
import simpy
from __future__ import division

In [73]:
servicetimes = []
import numpy as np

class Server(object):
    """Server with constant service times""" 
    
    def __init__(self, env):
        self.env = env
        self.server = simpy.Resource(env, capacity=personnelAmount) #amount of servers
    
    # select the shortest service time or the first in line
    def server_time(self): 
        #print(len(servicetimes))
        if servicePreference == 'shortestServiceFirst':
            tempvalue = np.amin(servicetimes)
            servicetimes.remove(tempvalue)
            return tempvalue
        if servicePreference == 'firstInFirstOut':
            tempvalue = servicetimes[0]
            del servicetimes[0]
            return tempvalue
        
    def serve(self):
        serve_time = self.server_time()
        yield self.env.timeout(serve_time)
    
    _server_time = server_time
    _variabele = "Hoi"
    
class Server2(Server):
    
    def server_time(self):
        return 2 

In [74]:
class Customer(object):
    """
    For each customer we record the arrivaltime, 
    the response time and the departure time.

    """
    def __init__(self, env, sv):
        self.env = env
        self.sv = sv
        self.time_in_line = 0  
        servicetimes.append(np.random.exponential(mu,1)) # mu service time  
        
        if serviceRateDistribution == 'ExponentialDistribution':
            servicetimes.append(np.random.exponential(mu,1)) # mu service time 
            
        if serviceRateDistribution == 'LongtailDistribution':
            random_number = np.random.random()
            if random_number > 1./10:
                server_time = np.random.exponential(mu,1) # mu service time 
            if random_number <= 1./10:
                server_time = np.random.exponential(2*mu,1) # mu service time 
            servicetimes.append(server_time)
    
        if serviceRateDistribution == 'BimodalDistribution':
            float_maker = 1000.
            random_number = np.random.random()
            if random_number > 1./10:
                server_time = np.random.exponential(mu,1)
            if random_number <= 1./10:              
                server_time = [1 + np.random.normal()/10.]
            servicetimes.append(server_time)                 
            
    def enqueue(self):
        #print("Arrived at %f" % (self.env.now))
        with self.sv.server.request() as request:
            self.time_in_line = self.env.now
            yield request
            self.time_in_line = self.env.now - self.time_in_line
            #print("Has waited at %f" % (self.time_in_line))
            datastorage.append(self.time_in_line)
            #print("Started serving at %f" % (self.env.now))
            yield self.env.process(self.sv.serve())
            #print("Finished serving at %f" % (self.env.now)) 

In [75]:
class CustomerGenerator(object):
    """Creates new customers"""
    def __init__(self,env,sv):
        self.env = env
        self.sv = sv
        self.max_customers = customerAmount
        
    def arrival_time(self):
        randomNumber = np.random.exponential(1./interarrivalTime,1) #1/lambda = interarravial time
        return randomNumber
    
    def run(self):
        customers = 0
        while customers < self.max_customers:
            arrival_time = self.arrival_time()
            yield self.env.timeout(arrival_time)
            
            customers += 1
            new_customer = Customer(self.env, self.sv)
            self.env.process(new_customer.enqueue())

In [76]:
datastorage = []
servicetimes = []

class QTExperiment(object):
    """ Easy interface to generate a queue """
    
    def __init__(self):
        self.env = simpy.Environment()
        self.server = Server(self.env)
        self.generator = CustomerGenerator(self.env, self.server)
        
    def run(self):
        self.env.process(self.generator.run())
        self.env.run()

In [77]:
datastorage = []
servicetimes = []

exp = QTExperiment().run()
#print(datastorage[100])
meandata = []
sigmadata = []

for i in range(200):
    meandata.append(np.mean(datastorage[i*int(samplingSpace) + int(halfMargin):(i+1)*int(samplingSpace) - int(halfMargin)]))
    sigmadata.append(np.std(datastorage[i*int(samplingSpace) + int(halfMargin):(i+1)*int(samplingSpace) - int(halfMargin)]))

print "mm"
print "sm"
print "ms"
print "ss"
print np.mean(meandata)
print np.std(meandata)
print np.mean(sigmadata)
print np.std(sigmadata)
print 
jump = 60
print np.mean(datastorage[jump*1200 + 100:jump*1200 + 1100])
print np.mean(datastorage[jump*2*1200 + 100:jump*2*1200 + 1100])
print np.mean(datastorage[jump*3*1200 + 100:jump*3*1200 + 1100])

meanMean8 = []
meanMean8 = meandata
print len(meanMean1)

mm
sm
ms
ss
0.8314433984291906
0.5979584055858967
0.8448518042833234
0.42348636781831994

1.0277570157384999
1.7031509824583482
1.0514164111527242
200


In [78]:
from scipy.stats import ttest_ind
statisticalResult = ttest_ind(meanMean1, meanMean8)
print statisticalResult
print np.var(meanMean1)

Ttest_indResult(statistic=21.397086781553273, pvalue=3.695792012598977e-68)
23.23558861368508


In [61]:
print meanMean1

[5.2354156308766795, 5.708778559452745, 6.675717397367133, 7.278554531467467, 13.93610187730939, 4.6370046804132565, 5.489015341396286, 5.6948351750318364, 10.140836157314022, 4.467964507555809, 4.317793091972761, 12.834871407692892, 5.7284790773919045, 7.793639453198077, 11.95418412101239, 7.189142545218499, 4.50893524471069, 7.088320426528939, 4.1556157668272204, 15.976034104008228, 5.462617742863084, 6.065308899927863, 12.67202026698689, 6.490612526839865, 9.810420124277123, 7.130748006114321, 5.373269981179266, 6.44618239701735, 7.186648018247477, 7.891559866865755, 4.671564750150275, 7.196326999671968, 4.575393638731235, 4.209187689074672, 5.188179512191295, 15.01072750650163, 7.663130265768639, 26.65028009560052, 2.444951842534836, 5.782530804855254, 5.385251446514325, 3.6570679429753, 7.732544332168524, 5.32721562192808, 6.42284525975876, 3.524763588949863, 7.373095275842024, 9.81051484032136, 10.346617103754724, 11.989137115483551, 8.100811750273431, 7.061127252119892, 15.16623

In [52]:
print meanMean2

[6.121078026911094, 4.381314836867936, 3.677845155809904, 1.807523827913298, 3.225962568906882, 3.4449579044813814, 5.346286123480895, 4.098391699413001, 2.9613537096295377, 2.6327986875455727, 4.001134659857802, 2.519359742186135, 2.0901516199708494, 4.398521946603427, 2.714705500963888, 2.798337720409054, 4.5236748588793, 2.992215764059509, 1.9392142174473501, 1.8909084051694371, 2.5439988329413152, 4.2927776766264945, 4.721736448291453, 2.1315690624427934, 2.153320878564771, 2.3219025886630043, 1.0715658530424899, 1.7409666758226376, 6.145018816053336, 2.3834817765616574, 1.5811616230626133, 8.827422647967937, 3.2483405044984646, 2.577872949925735, 1.533682200998177, 3.332557396703374, 5.976009931685268, 2.46493361979453, 3.378909136124719, 5.442655589273254, 2.4977156716730207, 1.476568171789484, 2.361576842246606, 3.820623559519005, 2.8283986698159316, 3.3492178162489328, 3.676408225448955, 4.573442184794821, 4.180817561531345, 3.006913677838951, 5.654612057251266, 5.7512373632978